**Table of contents**<a id='toc0_'></a>    
- [Voronoi](#toc1_)    
  - [Voronoi 2d](#toc1_1_)    
  - [Voronoi 6d](#toc1_2_)    
- [Gaussian PDF 1d](#toc2_)    
- [Guassian PDF function 2d](#toc3_)    
- [Guassian GMM 1d](#toc4_)    
- [Guassian GMM 1d with Unknow K cluster](#toc5_)    
- [Guassian GMM 2D](#toc6_)    
- [Sorting Sampling](#toc7_)    
- [Cspace Learning Path Prediction](#toc8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d, ConvexHull, Delaunay
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from shapely.geometry import Polygon as shapelyPolygon
from sklearn.mixture import GaussianMixture
import os
import time

np.set_printoptions(linewidth=1000, suppress=True, precision=2)
np.random.seed(42)
rsrc = os.environ["RSRC_DIR"]

# <a id='toc1_'></a>[Voronoi](#toc0_)

In [ ]:
def vrn2d_grid():
    x = np.linspace(0, 1, 10)
    y = np.linspace(0, 1, 10)
    points = np.meshgrid(x, y)
    points = np.array([points[0].ravel(), points[1].ravel()]).T
    vor = Voronoi(points)

    fig, ax = plt.subplots()
    voronoi_plot_2d(vor, ax)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect("equal")
    plt.show()


# vrn2d_grid()

## <a id='toc1_1_'></a>[Voronoi 2d](#toc0_)

In [ ]:
def vrn2d_random():
    points = np.random.uniform(0, 1, (10, 2))
    vor = Voronoi(points)

    fig, ax = plt.subplots()
    voronoi_plot_2d(vor, ax)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect("equal")
    plt.show()


# vrn2d_random()

## <a id='toc1_2_'></a>[Voronoi 6d](#toc0_)

In [ ]:
def vrn6d_random():
    rng = np.random.default_rng()
    points = rng.random((1000, 6))  # 10 s to make
    # points = rng.random((5000, 6))  # 106.54 s to make
    # points = rng.random((10000, 6))  # 255 s to make

    t0 = time.time()
    vor = Voronoi(points)
    t1 = time.time()
    print(f"Elapsed time: {t1 - t0:.2f} s")


# vrn6d_random()

In [ ]:
def __test_voronoi():
    rng = np.random.default_rng(9)
    points = rng.random((10, 2))
    print(f"> points: {points}")

    vor = Voronoi(points)
    print(f"> vor: {vor}")
    print(f"> vor.points: {vor.points}")
    print(f"> vor.vertices: {vor.vertices}")
    print(f"> vor.regions: {vor.regions}")

    # fig, ax = plt.subplots()
    # voronoi_plot_2d(vor, ax)
    # for i, p in enumerate(points):
    #     ax.text(p[0], p[1], f"p:{i}:{p}")
    # for i, v in enumerate(vor.vertices):
    #     ax.text(v[0], v[1], f"v:{i}:{v}")
    # for k, r in enumerate(vor.regions):
    #     if len(r) == 0:
    #         continue
    #     if -1 in r:
    #         continue
    #     vv = [vor.vertices[j] for j in r]
    #     vv = np.concatenate(vv)
    #     poly = Polygon([vor.vertices[j] for j in r], facecolor="none", edgecolor="r")
    #     ax.add_patch(poly)

    # ax.set_xlim(0, 1)
    # ax.set_ylim(0, 1)
    # ax.set_aspect("equal")
    # plt.show()

    p1 = shapelyPolygon(vor.vertices[vor.regions[4]])
    p2 = shapelyPolygon(vor.vertices[vor.regions[6]])
    p3 = shapelyPolygon(vor.vertices[vor.regions[7]])

    d = p1.distance(p2)
    ii = p1.intersection(p2)
    id = p1.intersects(p2)

    fig, ax = plt.subplots()
    voronoi_plot_2d(vor, ax)
    for i, p in enumerate(points):
        ax.text(p[0], p[1], f"p:{i}:{p}")
    for i, v in enumerate(vor.vertices):
        ax.text(v[0], v[1], f"v:{i}:{v}")
    p11 = Polygon(p1.exterior, facecolor="r", edgecolor="r")
    ax.add_patch(p11)
    p22 = Polygon(p2.exterior, facecolor="b", edgecolor="b")
    ax.add_patch(p22)
    p33 = Polygon(p3.exterior, facecolor="g", edgecolor="g")
    ax.add_patch(p33)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect("equal")
    plt.show()

    # vorrr = Voronoi(points, incremental=True)
    # print(f"> vorrr: {vorrr}")
    # vorrr.add_points(np.array([[0.5, 0.5]]))

    convx1 = ConvexHull(vor.vertices[vor.regions[4]])
    print(f"> convx1: {convx1}")
    eq1 = convx1.equations
    print(f"> eq1: {eq1}")

    fig, ax = plt.subplots()
    ax.plot(convx1.points[:, 0], convx1.points[:, 1], "o")
    p11 = Polygon(p1.exterior, facecolor="r", edgecolor="r")
    ax.add_patch(p11)
    for i in range(convx1.points.shape[0]):
        ax.axline(
            (convx1.points[i, 0], convx1.points[i, 1]),
            (
                convx1.points[(i + 1) % convx1.points.shape[0], 0],
                convx1.points[(i + 1) % convx1.points.shape[0], 1],
            ),
        )
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect("equal")
    plt.show()


# __test_voronoi()

# <a id='toc2_'></a>[Gaussian PDF 1d](#toc0_)

In [ ]:
def gaussian_pdf_1d():
    xm = 0
    xstd = 0.3
    npoints = 500
    xsample = np.random.normal(xm, xstd, (npoints,))
    y = 0 * np.arange(npoints)

    def normal_pdf(x, mu, sigma):
        return (1.0 / (np.sqrt(2 * np.pi) * sigma)) * np.exp(
            -0.5 * ((x - mu) / sigma) ** 2
        )

    def normal_pdf_grad(x, mu, sigma):
        f = normal_pdf(x, mu, sigma)
        return -(x - mu) / (sigma**2) * f

    xpdf = np.linspace(xm - 4 * xstd, xm + 4 * xstd, npoints)
    ygussian = normal_pdf(xpdf, xm, xstd)
    ypguassian = normal_pdf_grad(xpdf, xm, xstd)

    fig, ax = plt.subplots()
    ax.scatter(xsample, y, s=5, c="blue", label="Data points")
    ax.plot(xpdf, ygussian, c="red", label="Gaussian distribution PDF")
    ax.plot(xpdf, ypguassian, c="green", label="Gaussian PDF Gradient")
    ax.set_aspect("equal")
    ax.set_xlabel("X1")
    ax.set_ylabel("X2")
    ax.legend()
    ax.set_xlim(xm - 4 * xstd, xm + 4 * xstd)
    ax.set_ylim(-0.1, 2)
    plt.show()


# gaussian_pdf_1d()

# <a id='toc3_'></a>[Guassian PDF function 2d](#toc0_)

In [ ]:
def gaussian_pdf_2d():
    mu = np.array([0, 0])  # Mean vector
    Sigma = np.array([[0.3, 0.2], [0.2, 0.7]])  # Covariance must positive definite
    npoints = 500
    xsample = np.random.multivariate_normal(mu, Sigma, npoints)
    x1 = np.linspace(
        mu[0] - 4 * np.sqrt(Sigma[0, 0]),
        mu[0] + 4 * np.sqrt(Sigma[0, 0]),
        100,
    )
    x2 = np.linspace(
        mu[1] - 4 * np.sqrt(Sigma[1, 1]),
        mu[1] + 4 * np.sqrt(Sigma[1, 1]),
        100,
    )
    X1, X2 = np.meshgrid(x1, x2)
    pos = np.dstack((X1, X2))

    Sigma_inv = np.linalg.inv(Sigma)
    Sigma_det = np.linalg.det(Sigma)
    norm_const = 1.0 / (2 * np.pi * np.sqrt(Sigma_det))
    diff = pos - mu  # shape (...,2)
    exponent = np.einsum("...i,ij,...j", diff, Sigma_inv, diff)  # shape (...,)
    Y = norm_const * np.exp(-0.5 * exponent)  # PDF values on grid, shape (...,)

    # ----------------------------
    # Jacobian (gradient) of Y:  ∇f(x) = - f(x) * Sigma^{-1} (x - mu)
    # Vectorized computation:
    # ----------------------------
    # tmp = Sigma^{-1} @ (x-mu) for each grid point -> shape (...,2)
    tmp = np.einsum("ij,...j->...i", Sigma_inv, diff)  # shape (...,2)

    # gradient arrays
    grad = -Y[..., np.newaxis] * tmp  # shape (...,2)
    dY_dx1 = grad[..., 0]
    dY_dx2 = grad[..., 1]

    x0 = np.array([0.1, -0.05])
    f_x0 = norm_const * np.exp(-0.5 * (x0 - mu) @ Sigma_inv @ (x0 - mu))
    grad_x0 = -f_x0 * (Sigma_inv @ (x0 - mu))
    print("f(x0) =", f_x0)
    print("grad f(x0) =", grad_x0)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")
    ax.scatter(
        xsample[:, 0],
        xsample[:, 1],
        zs=0,
        zdir="z",
        s=5,
        c="blue",
        label="Data points",
    )
    # step = 8
    # ax.quiver(
    #     X1[::step, ::step],
    #     X2[::step, ::step],
    #     dY_dx1[::step, ::step],
    #     dY_dx2[::step, ::step],
    #     scale=50,  # adjust scale for arrow length visibility
    #     width=0.003,
    #     alpha=0.8,
    #     color="red",
    #     label="Gradient (Jacobian)",
    # )
    ax.plot_surface(X1, X2, Y, cmap="viridis", alpha=0.7)
    ax.set_xlabel("X1")
    ax.set_ylabel("X2")
    ax.set_zlabel("Probability Density")
    ax.set_title("2D Gaussian Distribution PDF")
    plt.show()


# gaussian_pdf_2d()

# <a id='toc4_'></a>[Guassian GMM 1d](#toc0_)

In [ ]:
def guassian_gmm_1d():
    # Generate synthetic 1D data
    np.random.seed(42)
    x = np.concatenate(
        [np.random.normal(-2, 0.5, 300), np.random.normal(3, 1.0, 300)]
    )
    x = x.reshape(-1, 1)

    K = 2
    gmm = GaussianMixture(n_components=K, covariance_type="full")
    gmm.fit(x)

    # Extract learned parameters
    pi = gmm.weights_  # mixture weights
    mu = gmm.means_.flatten()  # means
    sigma = np.sqrt(gmm.covariances_.flatten())  # standard deviations

    print("Mixture weights:", pi)
    print("Means:", mu)
    print("Sigmas:", sigma)

    def normal_pdf(x, mu, sigma):
        return (1.0 / (np.sqrt(2 * np.pi) * sigma)) * np.exp(
            -0.5 * ((x - mu) / sigma) ** 2
        )

    def gmm_pdf(x, pi, mu, sigma):
        total = np.zeros_like(x, dtype=float)
        for k in range(len(pi)):
            total += pi[k] * normal_pdf(x, mu[k], sigma[k])
        return total

    def gmm_pdf_grad(x, pi, mu, sigma):
        # dp/dx = sum_k pi_k * [- (x - mu_k) / sigma_k^2 ] * N(x | mu_k)
        grad = np.zeros_like(x, dtype=float)
        for k in range(len(pi)):
            pdf_k = normal_pdf(x, mu[k], sigma[k])
            grad += pi[k] * (-(x - mu[k]) / (sigma[k] ** 2)) * pdf_k
        return grad

    xgrid = np.linspace(x.min() - 1, x.max() + 1, 500).reshape(-1, 1)
    pdf = np.exp(gmm.score_samples(xgrid))  # full mixture PDF

    fig, ax = plt.subplots()
    ax.hist(x, bins=40, density=True, alpha=0.4, label="Data histogram")
    ax.plot(xgrid, pdf, linewidth=2, label="GMM PDF curve")
    ax.plot(
        xgrid,
        gmm_pdf_grad(xgrid, pi, mu, sigma),
        linewidth=2,
        label="GMM PDF Gradient",
    )

    # individual components
    for k in range(K):
        comp_pdf = (
            pi[k]
            * (1 / np.sqrt(2 * np.pi * sigma[k] ** 2))
            * np.exp(-((xgrid - mu[k]) ** 2) / (2 * sigma[k] ** 2))
        )
        ax.plot(xgrid, comp_pdf, linestyle="--", label=f"Component {k+1}")

    ax.set_xlabel("x")
    ax.set_ylabel("Density")
    ax.legend()
    plt.show()


# guassian_gmm_1d()

# <a id='toc5_'></a>[Guassian GMM 1d with Unknow K cluster](#toc0_)

In [ ]:
def guassian_gmm_1d_Kunkown():
    """
    Determine the optimal number of Gaussian components in a 1D GMM using AIC/BIC
    """
    random_state = np.random.RandomState(seed=1)

    X = np.concatenate(
        [
            random_state.normal(-1, 1.5, 350),
            random_state.normal(0, 1, 500),
            random_state.normal(3, 0.5, 150),
        ]
    ).reshape(-1, 1)

    # ------------------------------------------------------------
    # Learn the best-fit GaussianMixture models
    #  Here we'll use scikit-learn's GaussianMixture model. The fit() method
    #  uses an Expectation-Maximization approach to find the best
    #  mixture of Gaussians for the data

    # fit models with 1-10 components find the best number of components using AIC
    N = np.arange(1, 11)
    models = [None for i in range(len(N))]

    for i in range(len(N)):
        models[i] = GaussianMixture(N[i]).fit(X)

    # compute the AIC and the BIC
    AIC = [m.aic(X) for m in models]
    BIC = [m.bic(X) for m in models]

    # ------------------------------------------------------------
    # Plot the results
    #  We'll use three panels:
    #   1) data + best-fit mixture
    #   2) AIC and BIC vs number of components
    #   3) probability that a point came from each component

    fig = plt.figure(figsize=(5, 1.7))
    fig.subplots_adjust(left=0.12, right=0.97, bottom=0.21, top=0.9, wspace=0.5)

    # plot 1: data + best-fit mixture
    ax = fig.add_subplot(131)
    M_best = models[np.argmin(AIC)]

    x = np.linspace(-6, 6, 1000)
    logprob = M_best.score_samples(x.reshape(-1, 1))
    responsibilities = M_best.predict_proba(x.reshape(-1, 1))
    pdf = np.exp(logprob)
    pdf_individual = responsibilities * pdf[:, np.newaxis]

    ax.hist(X, 30, density=True, histtype="stepfilled", alpha=0.4)
    ax.plot(x, pdf, "-k")
    ax.plot(x, pdf_individual, "--k")
    ax.text(
        0.04, 0.96, "Best-fit Mixture", ha="left", va="top", transform=ax.transAxes
    )
    ax.set_xlabel("$x$")
    ax.set_ylabel("$p(x)$")

    # plot 2: AIC and BIC
    ax = fig.add_subplot(132)
    ax.plot(N, AIC, "-k", label="AIC")
    ax.plot(N, BIC, "--k", label="BIC")
    ax.set_xlabel("n. components")
    ax.set_ylabel("information criterion")
    ax.legend(loc=2)

    # plot 3: posterior probabilities for each component
    ax = fig.add_subplot(133)

    p = responsibilities
    p = p[:, (1, 0, 2)]  # rearrange order so the plot looks better
    p = p.cumsum(1).T

    ax.fill_between(x, 0, p[0], color="gray", alpha=0.3)
    ax.fill_between(x, p[0], p[1], color="gray", alpha=0.5)
    ax.fill_between(x, p[1], 1, color="gray", alpha=0.7)
    ax.set_xlim(-6, 6)
    ax.set_ylim(0, 1)
    ax.set_xlabel("$x$")
    ax.set_ylabel(r"$p({\rm class}|x)$")

    ax.text(-5, 0.3, "class 1", rotation="vertical")
    ax.text(0, 0.5, "class 2", rotation="vertical")
    ax.text(3, 0.3, "class 3", rotation="vertical")

    plt.show()


# guassian_gmm_1d_Kunkown()

# <a id='toc6_'></a>[Guassian GMM 2D](#toc0_)

In [ ]:
def guassian_gmm_2d():
    pass

# <a id='toc7_'></a>[GMM Image](#toc0_)

In [ ]:
def gmm_2dd_():
    n = 500
    mean1 = [0, 0]
    cov1 = [[1, 0.3], [0.3, 1]]
    mean2 = [4, 4]
    cov2 = [[1, -0.4], [-0.4, 1.2]]
    data1 = np.random.multivariate_normal(mean1, cov1, n)
    data2 = np.random.multivariate_normal(mean2, cov2, n)
    data = np.vstack([data1, data2])

    gmm = GaussianMixture(n_components=2, random_state=42)
    gmm.fit(data)

    print("Gaussian Mixture Model fitted successfully.")
    print("Estimated Means:\n", gmm.means_)
    print("\nEstimated Covariances:\n", gmm.covariances_)
    print("\nEstimated Component Weights:\n", gmm.weights_)

    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection="3d")

    print("3D plotting modules imported and figure setup for 3D plot.")
    x_min, x_max = data[:, 0].min() - 1, data[:, 0].max() + 1
    y_min, y_max = data[:, 1].min() - 1, data[:, 1].max() + 1
    x = np.linspace(x_min, x_max, 100)
    y = np.linspace(y_min, y_max, 100)
    X, Y = np.meshgrid(x, y)

    XY = np.vstack([X.ravel(), Y.ravel()]).T
    # Calculate the log-likelihood of each point on the meshgrid
    log_density = gmm.score_samples(XY)
    # Convert log-likelihoods to actual probability density function (PDF) values
    Z = np.exp(log_density)
    Z = Z.reshape(X.shape)

    ax.plot_surface(X, Y, Z, cmap="viridis", alpha=0.6)
    ax.scatter(
        data[:, 0],
        data[:, 1],
        np.zeros_like(data[:, 0]),
        c="blue",
        s=10,
        label="Original Data",
    )
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")
    ax.set_zlabel("Probability Density")
    ax.set_title("3D GMM PDF with Original Data Points")
    plt.tight_layout()
    plt.show()


# gmm_2dd_()

In [ ]:
# https://scipy-lectures.org/advanced/image_processing/auto_examples/plot_GMM.html
def gmm_image_segmentation():
    from scipy import ndimage

    n = 10
    l = 256
    im = np.zeros((l, l))
    points = l * np.random.random((2, n**2))
    im[(points[0]).astype(np.int32), (points[1]).astype(np.int32)] = 1
    im = ndimage.gaussian_filter(im, sigma=l / (4.0 * n))

    mask = (im > im.mean()).astype(np.float32)
    img = mask + 0.3 * np.random.randn(*mask.shape)

    hist, bin_edges = np.histogram(img, bins=60)
    bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

    classif = GaussianMixture(n_components=2)
    classif.fit(img.reshape((img.size, 1)))

    threshold = np.mean(classif.means_)
    binary_img = img > threshold

    plt.figure(figsize=(11, 4))
    plt.subplot(131)
    plt.imshow(img)
    plt.axis("off")
    plt.subplot(132)
    plt.plot(bin_centers, hist, lw=2)
    plt.axvline(0.5, color="r", ls="--", lw=2)
    plt.text(0.57, 0.8, "histogram", fontsize=20, transform=plt.gca().transAxes)
    plt.yticks([])
    plt.subplot(133)
    plt.imshow(binary_img, cmap=plt.cm.gray, interpolation="nearest")
    plt.axis("off")
    plt.subplots_adjust(
        wspace=0.02, hspace=0.3, top=1, bottom=0.1, left=0, right=1
    )
    plt.show()


# gmm_image_segmentation()

# <a id='toc7_'></a>[Sorting Sampling](#toc0_)

In [ ]:
def sorting_sampling_():
    xfake = np.linspace(0, 10, 100)
    yfake = np.sin(xfake)
    Xdata = np.vstack((xfake, yfake)).T
    v = np.array([1, 0])

    X = np.array(
        [
            [0.5, 1.5],
            [1.0, 0.5],
            [1.5, 1.0],
            [2.0, 2.5],
            [2.5, 1.0],
            [3.0, 3.5],
            [3.5, 3.0],
        ]
    )

    v = np.array([1, 1])
    v = v / np.linalg.norm(v)
    s = X @ v  # projection scalars
    order = np.argsort(s)
    X_sorted = X[order]

    print("Original X:")
    print(X)
    print("Projection scalars:")
    print(s)
    print("Sorted order indices:")
    print(order)
    print("Sorted X:")
    print(X_sorted)

    fig, ax = plt.subplots()
    ax.scatter(X[:, 0], X[:, 1], color="blue", label="Original Points")
    ax.scatter(X_sorted[:, 0], X_sorted[:, 1], color="red", label="Sorted Points")

    for i in range(X.shape[0]):
        ax.text(X[i, 0] + 0.1, X[i, 1], f"{i}", fontsize=12, color="blue")
        ax.text(X_sorted[i, 0], X_sorted[i, 1], f"{i}", fontsize=12, color="red")
    ax.plot(
        [0, v[0] * 6],
        [0, v[1] * 6],
        color="green",
        linestyle="--",
        label="Projection Direction",
    )
    ax.legend()
    ax.grid()
    ax.set_aspect("equal", "box")
    plt.show()


# sorting_sampling_()

# <a id='toc8_'></a>[Cspace Learning Path Prediction](#toc0_)

## Dataset 1

In [ ]:
dataset = np.load(os.path.join(rsrc, "cspace_dataset.npy"))
N_TRAIN = 1000
samples_id = np.random.choice(range(dataset.shape[0]), size=N_TRAIN, replace=False)
dataset_samples = dataset[samples_id]
X_train = dataset_samples[:, 0:2]
y = dataset_samples[:, 2]
# y_train = np.where(y <= 0, -1, +1)  # switch sign
y_train = np.where(y <= 0, +1, -1)  # switch sign
xfreegarantee = np.array([0.0, 0.0]).reshape(1, -1)

## Dataset 2

In [ ]:
def is_collision(x):
    R_OBS = 0.8
    return np.linalg.norm(x, axis=1) <= R_OBS


N_TRAIN = 600
dof = 2
X_train = np.random.uniform(-np.pi, np.pi, size=(N_TRAIN, dof))
# y_train = np.where(is_collision(X_train), -1, +1)
y_train = np.where(is_collision(X_train), +1, -1)
xfreegarantee = np.array([1, 1]).reshape(1, -1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import rbf_kernel
from scipy.sparse.csgraph import shortest_path
from sklearn.neighbors import NearestNeighbors

np.random.seed(42)


sigma = 0.5
K = rbf_kernel(X_train, X_train, gamma=1 / (2 * sigma**2))

alpha = np.zeros(N_TRAIN)

# simple online perceptron
epochs = 100
for _ in range(epochs):  # few epochs is enough
    for i in range(N_TRAIN):
        f_i = np.sum(alpha * y_train * K[:, i])
        if y_train[i] * f_i <= 0:
            alpha[i] += 1


def f_hat(X):
    Kx = rbf_kernel(X, X_train, gamma=1 / (2 * sigma**2))
    return Kx @ (alpha * y_train)


# check sign to flip the label for non-covex space application
if f_hat(xfreegarantee)[0] < 0:
    alpha *= -1

xges1 = np.array([[0.0, 0.0]])
xges2 = np.array([[1.0, 1.0]])
fges1 = f_hat(xges1)
print("f_hat at x = 0:", fges1)
fges2 = f_hat(xges2)
print("f_hat at x = 1:", fges2)

grid = 200
xs = np.linspace(-np.pi, np.pi, grid)
ys = np.linspace(-np.pi, np.pi, grid)
XX, YY = np.meshgrid(xs, ys)
XY = np.column_stack([XX.ravel(), YY.ravel()])
Z = f_hat(XY).reshape(grid, grid)
plt.figure(figsize=(6, 6))
plt.contourf(XX, YY, Z, levels=50, cmap="coolwarm")
plt.contour(XX, YY, Z, levels=[0], colors="black")
plt.gca().set_aspect("equal")
plt.title("Learned free-space scalar field f̂(x)")
plt.show()

In [ ]:
def soft_adjacency(X, f, sigma=0.6):
    W = rbf_kernel(X, X, gamma=1 / (2 * sigma**2))
    conf = 1 / (1 + np.exp(-f))  # sigmoid
    W *= np.minimum(conf[:, None], conf[None, :])
    np.fill_diagonal(W, 0)
    return W

                                                                                                                                           
def soft_adjacency_mid_penalty(X, f, sigma=0.6):
    W = rbf_kernel(X, X, gamma=1 / (2 * sigma**2))
    conf = 1 / (1 + np.exp(-f))  # sigmoid
    W *= np.minimum(conf[:, None], conf[None, :])

    # centers = (X_nodes[:, None, :] + X_nodes[None, :, :]) / 2
    # midpoints_flat = centers.reshape(-1, 2)
    # values = f_hat(midpoints_flat).reshape(len(X_nodes), len(X_nodes))
    # p = 1 / (1 + np.exp(-values))
    # W *= p
    # must optimize later for memory efficiency
    for i in range(len(X)):
        for j in range(i + 1, len(X)):
            m = 0.5 * (X[i] + X[j])
            p = 1 / (1 + np.exp(-f_hat(m[None])[0]))
            W[i, j] *= p
            W[j, i] *= p

    np.fill_diagonal(W, 0)
    return W


def soft_adjacency_local(X, f, sigma=0.6, k=15):
    N = len(X)
    W = np.zeros((N, N))

    nbrs = NearestNeighbors(n_neighbors=k + 1).fit(X)
    distances, indices = nbrs.kneighbors(X)

    conf = 1 / (1 + np.exp(-f))

    for i in range(N):
        for j, d in zip(indices[i][1:], distances[i][1:]):
            w = np.exp(-(d**2) / (2 * sigma**2))
            w *= np.minimum(conf[i], conf[j])
            W[i, j] = W[j, i] = w

    return W


def soft_adjacency_radius(X, f, sigma=0.6, r_conn=0.6):
    N = len(X)
    W = np.zeros((N, N))

    nbrs = NearestNeighbors(radius=r_conn).fit(X)
    indices = nbrs.radius_neighbors(X, return_distance=False)

    conf = 1 / (1 + np.exp(-f))

    for i in range(N):
        for j in indices[i]:
            if j == i:
                continue
            d = np.linalg.norm(X[i] - X[j])
            w = np.exp(-(d**2) / (2 * sigma**2))
            w *= min(conf[i], conf[j])
            W[i, j] = w
            W[j, i] = w

    return W

In [ ]:
N_NODES = 200
X_nodes = np.random.uniform(-np.pi, np.pi, size=(N_NODES, 2))
f_nodes = f_hat(X_nodes)

# confidence threshold filter
tau = 0.5
mask = f_nodes > tau
X_nodes = X_nodes[mask]
f_nodes = f_nodes[mask]

# W = soft_adjacency(X_nodes, f_nodes)
W = soft_adjacency_mid_penalty(X_nodes, f_nodes)
print(f"Shape of W of sample free node before start-goal {W.shape}")
# W = soft_adjacency_local(X_nodes, f_nodes, k=15)
# W = soft_adjacency_radius(X_nodes, f_nodes, r_conn=0.2)

# estimate shortest path from start to goal
start = np.array([-2.5, -2.5])
goal = np.array([2.5, 2.5])
X_all = np.vstack([start, goal, X_nodes])
f_all = np.concatenate([f_hat(start[None])[0:1], f_hat(goal[None])[0:1], f_nodes])
# W_all = soft_adjacency(X_all, f_all)
W_all = soft_adjacency_mid_penalty(X_all, f_all)
print(f"Shape of W of all nodes with start-goal {W_all.shape}")
C = 1 / (W_all + 1e-6)  # convert similarity → cost

# debug
print(f"W_all", W_all)
print(f"C", C)

dist, pred = shortest_path(C, return_predecessors=True)
path = []
j = 1  # goal index
while j != -9999:
    path.append(j)
    j = pred[0, j]
path = path[::-1]

pathlength = 0.0
for i in range(len(path) - 1):
    pathlength += np.linalg.norm(X_all[path[i + 1]] - X_all[path[i]])
print(f"Path length: {pathlength:.2f}")

path_xy = X_all[path]
plt.figure(figsize=(6, 6))
plt.contourf(XX, YY, Z, levels=50, cmap="coolwarm", alpha=0.4)
theta = np.linspace(0, 2 * np.pi, 200)
# plt.plot(R_OBS * np.cos(theta), R_OBS * np.sin(theta), "k")
plt.scatter(X_nodes[:, 0], X_nodes[:, 1], s=5, c="blue")
plt.plot(path_xy[:, 0], path_xy[:, 1], "k-", linewidth=2)
plt.scatter(*start, c="green", s=80)
plt.scatter(*goal, c="red", s=80)
plt.gca().set_aspect("equal")
plt.title("Path via learned free-space connectivity (no collision checks)")
plt.show()

In [ ]:
# estimate shortest path from start to goal
import time
q1 = np.array([-1.0, 2.5])
q2 = np.array([1.0, 2.5])
q3 = np.array([0.15, 0.60])
q4 = np.array([2.5, 1.5])
q5 = np.array([-2.5, -1.5])
q6 = np.array([2.40, -0.4])
q7 = np.array([-2.0, 2.5])
q8 = np.array([1.0, -2.0])
q9 = np.array([-3.0, 0.0])
q10 = np.array([-3.0, 2.5])
# sgpairs = {
#     0: (np.array([-3.0, 0.0]), np.array([-3.0, 2.5])),
#     1: (np.array([0.0, 2.0]), np.array([2.5, -2.5])),
#     2: (np.array([0.0, 2.0]), np.array([0.0, -2.5])),
# }
sgpairs = {
    0: (q1, q2),
    1: (q3, q4),
    2: (q5, q6),
    3: (q7, q8),
    4: (q9, q10),
}

pathsave = {x: [] for x in sgpairs}
pathlengthsave = {x: 0.0 for x in sgpairs}
X_allsave = {x: None for x in sgpairs}

for key in sgpairs:
    p1s = time.time()
    start, goal = sgpairs[key]
    print(f"Processing start-goal pair {key}: start={start}, goal={goal}")
    X_all = np.vstack([start, goal, X_nodes])
    X_allsave[key] = X_all
    f_all = np.concatenate(
        [f_hat(start[None])[0:1], f_hat(goal[None])[0:1], f_nodes]
    )
    W_all = soft_adjacency_mid_penalty(X_all, f_all)
    C = 1 / (W_all + 1e-6)  # convert similarity → cost
    p1e = time.time()
    print(f"  Adjacency matrix computation time: {p1e - p1s:.2f} s")

    # save for debugging
    # savep = os.path.join(rsrc, f"W_all_{key}.txt")
    # np.savetxt(savep, W_all, delimiter=",", fmt="%.2f")

    p2s = time.time()
    dist, pred = shortest_path(C, return_predecessors=True)
    path = []
    j = 1  # goal index
    while j != -9999:
        path.append(j)
        j = pred[0, j]
    path = path[::-1]
    pathsave[key] = path
    p2e = time.time()
    print(f"  Shortest path computation time: {p2e - p2s:.2f} s")

    pathlength = 0.0
    for i in range(len(path) - 1):
        pathlength += np.linalg.norm(X_all[path[i + 1]] - X_all[path[i]])
    pathlengthsave[key] = pathlength

fig, axes = plt.subplots(1, 5, figsize=(30, 6))
for key in sgpairs:
    path_xy = X_allsave[key][pathsave[key]]
    start, goal = sgpairs[key]
    axes[key].set_title(
        f"Est.Path {key} length: {pathlengthsave[key]:.2f}", fontsize=14
    )
    axes[key].contourf(XX, YY, Z, levels=50, cmap="coolwarm", alpha=0.4)
    axes[key].scatter(X_nodes[:, 0], X_nodes[:, 1], s=6, c="blue")
    axes[key].plot(path_xy[:, 0], path_xy[:, 1], "k-", linewidth=2)
    axes[key].scatter(*start, c="green", s=80)
    axes[key].scatter(*goal, c="red", s=80)
    axes[key].set_aspect("equal")
plt.show()

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

sigma = 0.5
model = SVC(kernel="rbf", gamma=1 / (2 * sigma**2), C=1.0)
ynnn_train = np.where(y_train == -1, 1, -1)

model.fit(X_train, ynnn_train)
fhater = model.decision_function
# N = 10000000, 6dof trained in 3min on laptop

In [ ]:
dof = 2
xges = np.array([[0.0] * dof])
xges2 = np.array([[1.0] * dof])
y_pred = model.predict(xges)
y_pred2 = model.predict(xges2)
fges = fhater(xges)
fges2 = fhater(xges2)
print("SVM prediction at xges (0s):", y_pred)
print("SVM prediction at xges2 (1s):", y_pred2)
print("SVM decision function at xges (0s):", fges)
print("SVM decision function at xges2 (1s):", fges2)

xt = np.array([[0, 0]])
ft = fhater(xt)
print("SVM decision function at test point (0,0):", ft)
xtdelta = np.array([[0.1, 0.1]])
ftdelta = fhater(xtdelta)
print("SVM decision function at test point (0.1, 0.1):", ftdelta)

In [ ]:
supvecs = model.support_vectors_
supvecs_labels = y_train[model.support_]
supvecs_free = supvecs[supvecs_labels == +1]
supvecs_cols = supvecs[supvecs_labels == -1]

In [ ]:
def soft_adjacency_mid_penalty(X, f, sigma=0.6):
    W = rbf_kernel(X, X, gamma=1 / (2 * sigma**2))
    conf = 1 / (1 + np.exp(-f))  # sigmoid
    W *= np.minimum(conf[:, None], conf[None, :])

    # centers = (X_nodes[:, None, :] + X_nodes[None, :, :]) / 2
    # midpoints_flat = centers.reshape(-1, 2)
    # values = f_hat(midpoints_flat).reshape(len(X_nodes), len(X_nodes))
    # p = 1 / (1 + np.exp(-values))
    # W *= p
    # must optimize later for memory efficiency
    for i in range(len(X)):
        for j in range(i + 1, len(X)):
            m = 0.5 * (X[i] + X[j])
            p = 1 / (1 + np.exp(-fhater(m[None])[0]))
            W[i, j] *= p
            W[j, i] *= p

    np.fill_diagonal(W, 0)
    return W


N_NODES = 200
X_nodes = np.random.uniform(-np.pi, np.pi, size=(N_NODES, 2))
X_nodes = supvecs_free
# X_nodes = supvecs_cols
f_nodes = fhater(X_nodes)

# confidence threshold filter
# tau = 0.5
# mask = f_nodes > tau
# X_nodes = X_nodes[mask]
# f_nodes = f_nodes[mask]


# estimate shortest path from start to goal
start = np.array([-2.5, -2.5])
goal = np.array([2.5, 2.5])
X_all = np.vstack([start, goal, X_nodes])
f_all = np.concatenate([fhater(start[None])[0:1], fhater(goal[None])[0:1], f_nodes])
# W_all = soft_adjacency(X_all, f_all)
W_all = soft_adjacency_mid_penalty(X_all, f_all)
print(f"Shape of W of all nodes with start-goal {W_all.shape}")
C = 1 / (W_all + 1e-6)  # convert similarity → cost

In [ ]:
dist, pred = shortest_path(C, return_predecessors=True)
path = []
j = 1  # goal index
while j != -9999:
    path.append(j)
    j = pred[0, j]
path = path[::-1]

pathlength = 0.0
for i in range(len(path) - 1):
    pathlength += np.linalg.norm(X_all[path[i + 1]] - X_all[path[i]])
print(f"Path length: {pathlength:.2f}")

pathq = X_all[path]
print(f"Path indices: {path}")
print(f"Path length: {pathlength:.2f}")
print(f"Path configurations:\n{pathq}")

plt.figure(figsize=(6, 6))
plt.contourf(XX, YY, Z, levels=50, cmap="coolwarm", alpha=0.4)
plt.scatter(X_nodes[:, 0], X_nodes[:, 1], s=5, c="blue")
plt.plot(pathq[:, 0], pathq[:, 1], "k-", linewidth=2)
plt.scatter(*start, c="green", s=80)
plt.scatter(*goal, c="red", s=80)
plt.gca().set_aspect("equal")
plt.title("Path via learned free-space connectivity (no collision checks)")
plt.show()

In [ ]:
grid = 200
xs = np.linspace(-np.pi, np.pi, grid)
ys = np.linspace(-np.pi, np.pi, grid)
XX, YY = np.meshgrid(xs, ys)
XY = np.column_stack([XX.ravel(), YY.ravel()])
Z = model.predict(XY).reshape(grid, grid)

plt.figure(figsize=(6, 6))
plt.contourf(XX, YY, Z, levels=50, cmap="coolwarm")
plt.contour(XX, YY, Z, levels=[0], colors="black")
plt.plot(
    supvecs_free[:, 0],
    supvecs_free[:, 1],
    "ro",
    markersize=5,
    label="Support Vectors (+1)",
)
plt.plot(
    supvecs_cols[:, 0],
    supvecs_cols[:, 1],
    "bo",
    markersize=5,
    label="Support Vectors (-1)",
)
# plt.plot(sup_vecs[:, 0], sup_vecs[:, 1], "ko", markersize=3, label="Support Vectors")
plt.gca().set_aspect("equal")
plt.title("Learned free-space scalar field f̂(x)")
plt.show()

In [ ]:
grid = 200
xs = np.linspace(-np.pi, np.pi, grid)
ys = np.linspace(-np.pi, np.pi, grid)
XX, YY = np.meshgrid(xs, ys)
XY = np.column_stack([XX.ravel(), YY.ravel()])
Z = fhater(XY).reshape(grid, grid)
plt.figure(figsize=(6, 6))
plt.contourf(XX, YY, Z, levels=50, cmap="coolwarm")
plt.contour(XX, YY, Z, levels=[0], colors="black")
plt.plot(
    supvecs_free[:, 0],
    supvecs_free[:, 1],
    "ro",
    markersize=5,
    label="Support Vectors (+1)",
)
plt.plot(
    supvecs_cols[:, 0],
    supvecs_cols[:, 1],
    "bo",
    markersize=5,
    label="Support Vectors (-1)",
)
plt.gca().set_aspect("equal")
plt.title("Learned free-space scalar field f̂(x)")
plt.show()

In [ ]:
q1 = np.array([-1.0, 2.5])
q2 = np.array([1.0, 2.5])
q3 = np.array([0.15, 0.60])
q4 = np.array([2.5, 1.5])
q5 = np.array([-2.5, -1.5])
q6 = np.array([2.40, -0.4])
q7 = np.array([-2.0, 2.5])
q8 = np.array([1.0, -2.0])
q9 = np.array([-3.0, 0.0])
q10 = np.array([-3.0, 2.5])

In [ ]:
pp = np.linspace(q1, q2, 10)
fp = fhater(pp)
print("SVM decision function along line from q1 to q2:", fp)

In [ ]:
grid = 200
xs = np.linspace(-np.pi, np.pi, grid)
ys = np.linspace(-np.pi, np.pi, grid)
XX, YY = np.meshgrid(xs, ys)
XY = np.column_stack([XX.ravel(), YY.ravel()])
Z = fhater(XY).reshape(grid, grid)

plt.figure(figsize=(6, 6))
plt.plot([q1[0], q2[0]], [q1[1], q2[1]], 'k-', linewidth=2)
plt.plot(pp[:,0], pp[:,1], 'ro--')
plt.contourf(XX, YY, Z, levels=50, cmap="coolwarm")
plt.contour(XX, YY, Z, levels=[0], colors="black")
plt.gca().set_aspect("equal")
plt.title("Learned free-space scalar field f̂(x)")
plt.show()

In [ ]:
def mapping(signal, realmin, realmax):
    sigmin = np.min(signal)
    sigmax = np.max(signal)
    scaled = (signal - sigmin) / (sigmax - sigmin)
    mapped = realmin + scaled * (realmax - realmin)
    return mapped


grid = 200
xs = np.linspace(-np.pi, np.pi, grid)
ys = np.linspace(-np.pi, np.pi, grid)
XX, YY = np.meshgrid(xs, ys)
XY = np.column_stack([XX.ravel(), YY.ravel()])
Z = fhater(XY).reshape(grid, grid)
Z = 1 / (1 + np.exp(-Z))  # sigmoid
Z = mapping(Z, -1, 1)
print(np.max(Z), np.min(Z))

plt.figure(figsize=(6, 6))
plt.plot([q1[0], q2[0]], [q1[1], q2[1]], "k-", linewidth=2)
plt.plot(pp[:, 0], pp[:, 1], "ro--")
plt.contourf(XX, YY, Z, levels=50, cmap="coolwarm")
plt.contour(XX, YY, Z, levels=[0], colors="black")
plt.gca().set_aspect("equal")
plt.title("Learned free-space scalar field f̂(x)")
plt.show()